# **Ejemplo de uso de un Notebbok de AI Platform Notebooks**
En este ejmplo se realiza del procesamiento del dataset publico **_Census_** y se muestra como **_AI Platform Notebboks_** puede interactuar con otros servicios de **GCP**. <br><br>


1. **Cargar Census Datset (desde Bigquery)**
    * Cargar mediante la libreria de Bigquery para python.
    * Cargar mediante las magias de Notebboks. [1.2](#1.2)
    
2. **Preprocesar dataset**
    * Eliminar filas con valores corruptos (Nan, None).
    * Transformar variables categoricas.
    * Transformar variables categoricas ordinales.
    
3. **Cargar dataset a Google Cloud Storage (GCS)**
    * Dividir dataset en Train, Validation, Test.
    * Guardar Dataset en local.
    * Guardar en GCS usando **gsutil**.

4. **Entrenar modelo en local**
    * Realizar inferencias

# Referencias
* [Bigquery client python](https://cloud.google.com/bigquery/docs/reference/libraries)
* [Magias Bigquery](https://cloud.google.com/bigquery/docs/visualize-jupyter?hl=es-419)

In [ ]:
# instalar librerias
!pip install -r requirements.txt

In [1]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
import os

In [2]:
local = True
GPU = True

In [3]:
if local:
    import os
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/macbook/Desktop/Rabbit/key/key.json"

if GPU:
    !nvidia-smi

zsh:1: command not found: nvidia-smi


<img src="https://miro.medium.com/max/1000/1*Ie1xfpPGkGKn8W3im8lr7g.png" title="BigQuery Logo"/>

## Cargar Census Datset (desde Bigquery)
### Cargar mediante la libreria de Bigquery para python

In [17]:
# Instancio cliente de BigQuery
client = bigquery.Client()

query = """
    SELECT
    *
    FROM
    `bigquery-public-data.ml_datasets.census_adult_income`
    LIMIT
    10;
"""
query_job = client.query(query)  # Make an API request.

print("The query data:")
for i,row in enumerate(query_job):
    # Row values can be accessed by field name or index.
    print("name={}, count={}".format(row[0], row[:]))

The query data:
name=39, count=(39, ' Private', 297847, ' 9th', 5, ' Married-civ-spouse', ' Other-service', ' Wife', ' Black', ' Female', 3411, 0, 34, ' United-States', ' <=50K')
name=72, count=(72, ' Private', 74141, ' 9th', 5, ' Married-civ-spouse', ' Exec-managerial', ' Wife', ' Asian-Pac-Islander', ' Female', 0, 0, 48, ' United-States', ' >50K')
name=45, count=(45, ' Private', 178215, ' 9th', 5, ' Married-civ-spouse', ' Machine-op-inspct', ' Wife', ' White', ' Female', 0, 0, 40, ' United-States', ' >50K')
name=31, count=(31, ' Private', 86958, ' 9th', 5, ' Married-civ-spouse', ' Exec-managerial', ' Wife', ' White', ' Female', 0, 0, 40, ' United-States', ' <=50K')
name=55, count=(55, ' Private', 176012, ' 9th', 5, ' Married-civ-spouse', ' Tech-support', ' Wife', ' White', ' Female', 0, 0, 23, ' United-States', ' <=50K')
name=30, count=(30, ' Private', 61272, ' 9th', 5, ' Married-civ-spouse', ' Machine-op-inspct', ' Wife', ' White', ' Female', 0, 0, 40, ' Portugal', ' <=50K')
name=46

<a id=’1.2’></a>
### Cargar mediante las magias de Notebboks
ejecutar consulta a bigquery para extraer el dataset census y guardarlo en la variable df.


In [6]:
%load_ext google.cloud.bigquery

In [7]:
%%bigquery df
SELECT
  age,
  workclass,
  native_country,
  marital_status,
  education_num,
  occupation,
  race,
  hours_per_week,
  income_bracket,
  CASE
    WHEN MOD(functional_weight, 10) < 8 THEN 'training'
    WHEN MOD(functional_weight, 10) = 8 THEN 'evaluation'
    WHEN MOD(functional_weight, 10) = 9 THEN 'prediction'
  END AS dataframe
FROM
  `bigquery-public-data.ml_datasets.census_adult_income`


Downloading: 100%|██████████| 32561/32561 [00:03<00:00, 8147.04rows/s] 


In [8]:
print(f"Census Dataset tamaño ------> {len(df)}")
df.head()

Census Dataset tamaño ------> 32561


,age,workclass,native_country,marital_status,education_num,occupation,race,hours_per_week,income_bracket,dataframe
0,39,Private,United-States,Married-civ-spouse,5,Other-service,Black,34,<=50K,training
1,72,Private,United-States,Married-civ-spouse,5,Exec-managerial,Asian-Pac-Islander,48,>50K,training
2,45,Private,United-States,Married-civ-spouse,5,Machine-op-inspct,White,40,>50K,training
3,31,Private,United-States,Married-civ-spouse,5,Exec-managerial,White,40,<=50K,evaluation
4,55,Private,United-States,Married-civ-spouse,5,Tech-support,White,23,<=50K,training


## Preprocesar dataset
### Eliminar filas con valores corruptos (Nan, None)

In [9]:
df = df.dropna()
print(f"Census Dataset tamaño ------> {len(df)}")

Census Dataset tamaño ------> 32561


### Transformar variables categoricas

In [10]:
# explorar variables categoricas
categorical_key = ["workclass", "native_country", "marital_status", "occupation", "race"]
for key_col in categorical_key:
    print(f"-------- {key_col} ----------")
    ls = list(df[key_col].unique())
    print(f"{key_col}: Number of unique values ------>  {len(ls)}")
    print(ls)
    print("----------------------------------------------\n")

-------- workclass ----------
workclass: Number of unique values ------>  9
[' Private', ' Self-emp-inc', ' Local-gov', ' ?', ' Self-emp-not-inc', ' Federal-gov', ' State-gov', ' Without-pay', ' Never-worked']
----------------------------------------------

-------- native_country ----------
native_country: Number of unique values ------>  42
[' United-States', ' Portugal', ' Yugoslavia', ' Hong', ' Haiti', ' Dominican-Republic', ' Mexico', ' Guatemala', ' Cuba', ' ?', ' Japan', ' Laos', ' Philippines', ' Germany', ' Italy', ' El-Salvador', ' Vietnam', ' Canada', ' Puerto-Rico', ' Honduras', ' Jamaica', ' South', ' Trinadad&Tobago', ' Greece', ' Peru', ' Iran', ' Columbia', ' Scotland', ' China', ' Nicaragua', ' England', ' Poland', ' Outlying-US(Guam-USVI-etc)', ' Taiwan', ' Hungary', ' India', ' Ecuador', ' Thailand', ' Cambodia', ' France', ' Ireland', ' Holand-Netherlands']
----------------------------------------------

-------- marital_status ----------
marital_status: Number of 

In [11]:
# Usar One Hot Encoding para convertir las categorias a numeros
for key_col in categorical_key:
    # one hot encoding
    one_hot = pd.get_dummies(df[key_col], prefix=key_col)
    # remove column
    df = df.drop(key_col,axis = 1)
    # add one hot encoding
    df = df.join(one_hot)

In [12]:
df

,age,education_num,hours_per_week,income_bracket,dataframe,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,...,occupation_ Prof-specialty,occupation_ Protective-serv,occupation_ Sales,occupation_ Tech-support,occupation_ Transport-moving,race_ Amer-Indian-Eskimo,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White
0,39,5,34,<=50K,training,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,72,5,48,>50K,training,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
2,45,5,40,>50K,training,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,31,5,40,<=50K,evaluation,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
4,55,5,23,<=50K,training,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,20,10,30,<=50K,training,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
32557,48,10,40,>50K,training,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
32558,22,10,43,<=50K,training,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1
32559,20,10,20,<=50K,training,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1


### Transformar variables categoricas ordinales

In [13]:
# explorar variables categoricas ordinales
categorical_key = ["age", "education_num", "hours_per_week"]
for key_col in categorical_key:
    print(f"-------- {key_col} ----------")
    ls = list(np.sort(df[key_col].unique()))
    print(f"{key_col}: Number of unique values ------>  {len(ls)}")
    print(ls)
    print("----------------------------------------------\n")

-------- age ----------
age: Number of unique values ------>  73
[17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 90]
----------------------------------------------

-------- education_num ----------
education_num: Number of unique values ------>  16
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
----------------------------------------------

-------- hours_per_week ----------
hours_per_week: Number of unique values ------>  94
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 70, 72, 73, 74, 75, 76, 77, 78, 80, 81, 82, 84, 85, 86

In [14]:
# normalizar variables ordinales
for key_col in categorical_key:
    ls = np.sort(df[key_col].unique())
    min = ls.min()
    max = ls.max()
    normalize = lambda x: (x-min)/(max-min)
    df[key_col] = df[key_col].apply(normalize)


In [15]:
"""
transformar etiquetas 

<=50K ----> 1
>50K -----> 0
"""
df["income_bracket"] = df["income_bracket"].apply(lambda x: 1 if x==" <=50K" else 0)
df

,age,education_num,hours_per_week,income_bracket,dataframe,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,...,occupation_ Prof-specialty,occupation_ Protective-serv,occupation_ Sales,occupation_ Tech-support,occupation_ Transport-moving,race_ Amer-Indian-Eskimo,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White
0,0.301370,0.266667,0.336735,1,training,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,0.753425,0.266667,0.479592,0,training,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
2,0.383562,0.266667,0.397959,0,training,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,0.191781,0.266667,0.397959,1,evaluation,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
4,0.520548,0.266667,0.224490,1,training,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,0.041096,0.600000,0.295918,1,training,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
32557,0.424658,0.600000,0.397959,0,training,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
32558,0.068493,0.600000,0.428571,1,training,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1
32559,0.041096,0.600000,0.193878,1,training,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1


<img src="https://sysblinders.com/wp-content/uploads/2020/08/google-cloud-storage.png" title="GCS Logo"/>

## Cargar dataset a Google Cloud Storage (GCS)
### Dividir dataset en Train, Validation, Test

In [16]:
# filtrar por la etiqueta el dataset preprocesado
df_train = df[df["dataframe"]=="training"]
#df_train = df_train.drop(["dataframe","income_bracket"],axis=1)

df_validation = df[df["dataframe"]=="evaluation"]
#df_validation = df_validation.drop(["dataframe","income_bracket"],axis=1)

df_test = df[df["dataframe"]=="prediction"]
#df_test = df_test.drop(["dataframe","income_bracket"],axis=1)

### Guardar Dataset en local

In [18]:
PATH_DATASET = "Dataset"
if not os.path.exists(PATH_DATASET):
    os.makedirs(PATH_DATASET)

df_train.to_csv("Dataset/train_census.csv",index=False)
df_validation.to_csv("Dataset/validation_census.csv",index=False)
df_test.to_csv("Dataset/test_census.csv",index=False)

### Guardar en GCS

In [19]:
%%bash
PATH_DATASET=Dataset
BUCKET=datasets-rabbit
gsutil cp -r ${PATH_DATASET} gs://${BUCKET}

Copying file://Dataset/validation_census.csv [Content-Type=text/csv]...
Copying file://Dataset/test_census.csv [Content-Type=text/csv]...
Copying file://Dataset/train_census.csv [Content-Type=text/csv]...
-
Operation completed over 3 objects/6.7 MiB.                                      


<img src="https://cdn.freelogovectors.net/wp-content/uploads/2018/07/tensorflow-logo.png" title="GCS Logo"/>

## Entrenar modelo en local

In [4]:
%%bash
BUCKET=datasets-rabbit
OUTPUT_DIR=Model
python task.py \
    --BUCKET=${BUCKET} \
    --FILE_TRAIN=Dataset/train_census.csv \
    --FILE_VAL=Dataset/validation_census.csv \
    --OUTPUT_DIR=${OUTPUT_DIR} \
    --DENSE_UNITS=32 \
    --EPOCHS=10 \
    --BATCH_SIZE=256 

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 81)]              0         
_________________________________________________________________
dense (Dense)                (None, 32)                2624      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 3,713
Trainable params: 3,713
Non-trainable params: 0
___________________________________________________________

### Realizar inferencias

In [5]:
from tensorflow.keras.models import load_model
model = load_model("Model")

In [12]:
path_dataset = "Dataset/test_census.csv"
df = pd.read_csv(path_dataset)
x = df.drop(["income_bracket","dataframe"],axis=1)
y = list(df["income_bracket"])

data_to_predict = x.iloc[0:].values.tolist()

In [13]:
response = model.predict(data_to_predict)

In [14]:
# transform probabilities to 1 or 0 values
predicted = [1 if r[0]>=0.5 else 0 for r in response]

In [15]:
pd.DataFrame({"predicted":predicted, "real":y})

,predicted,real
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
...,...,...
3137,1,1
3138,1,1
3139,1,1
3140,1,1


In [16]:
err = np.sum(np.array(predicted)-np.array(y))
N = len(x) 
print(f"Predicciones correctas -------->    {N-err} de {N}")
print(f"Acierto en test ---------------->   {(N-err)/N*100:.2f}%")

Predicciones correctas -------->    2952 de 3142
Acierto en test ---------------->   93.95%
